# Setup

Tutorial for reddit scraping: https://www.geeksforgeeks.org/scraping-reddit-using-python/

In [1]:
import praw
import pandas as pd
from praw.models import MoreComments
# from tqdm.notebook import tqdm
from tqdm import tqdm


# ids for scraping (from christians setup)
client_id = 'Ut5UgaAMOEWBELtYRWnw0g'
client_secret = '5xGs1w6mav5Ke685afpG28Q8nfusmg'
user_agent = 'polarity search'

# Scraping

First we initialize a read-only instance. A read-only instance can only scrape publicly available information and cannot upvote or otherwise interact like users can.

In [2]:
# Read-only instance
reddit_read_only = praw.Reddit(client_id=client_id,         # your client id
                               client_secret=client_secret,      # your client secret
                               user_agent=user_agent)        # your user agent

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


## Getting comments on a specific post

This code scrapes over the comments of a specified post. It looks only at the top-level comments (none of the replies to comments).

In [3]:
def scrape_post(url, n=100):
    '''given a url:
     - scrapes n number of top-level comments
     - comment author (username) and comment body ()
     - outputs a pandas dataframe'''

    # Creating a submission object
    submission = reddit_read_only.submission(url=url)
    
    # should get all top level comments on the post
    #if all_comments==True:
    #    submission.comments.replace_more(limit=None)

    post_authors = []
    post_comments = []

    # specifying how many times we should "load more comments"
    limit = round(n/20) # n/20 ensures we hit load more enough times to scrape the number of comments we want 
    submission.comments.replace_more(limit=limit, threshold=0)

    count = 0 # tracking how many comments have been scraped
    for comment in submission.comments: # iterates only over top level comments
        # stops counting when n amount of comments have been scraped
        if count == n:
            break

        if type(comment) == MoreComments:
            continue

        post_authors.append(comment.author)
        post_comments.append(comment.body)

        count += 1

    post_dict = {'author': post_authors, 'comment': post_comments}
    post_df = pd.DataFrame(post_dict)
    
    return post_df

### Examples of how scraping a post works

In [ ]:
# scraping a post without much content - takes <1 second
# there are 12 top level comments
df = scrape_post("https://www.reddit.com/r/MaraudersGame/comments/ylxsq4/marauders_be_like/")

print(df.shape)
df.head()

In [ ]:
# scraping a decently sized post (function default scrapes 100 comments) - takes ~10 seconds
df = scrape_post("https://www.reddit.com/r/politics/comments/1092xhl/the_american_public_no_longer_believes_the/")

print(df.shape)
df.head()

In [ ]:
# scraping 200 comments from a decently sized post (post has 3.8k comments) - takes ~15 seconds
df = scrape_post("https://www.reddit.com/r/politics/comments/1092xhl/the_american_public_no_longer_believes_the/", n=200)

print(df.shape)
df.head()

In [ ]:
# aidan: idk what below line was so i just commented it out
# praw.models.reddit.submission.Submission?

## Getting top month posts on specified subreddit
This code grabs the top 100 posts of the past month and saves various information on them into a dictionary

In [4]:
def scrape_top_month(subreddit, ppsr=100):
    # specifying subreddit
    subreddit = reddit_read_only.subreddit(subreddit)

    # Specifying to look at top posts of the current month
    posts = subreddit.top("month", limit=ppsr)

    # Initializing dictionary to save post data to
    posts_dict = {"Title": [], "Post Text": [],
                  "ID": [], "Score": [],
                  "Total Comments": [], "Post URL": [], 'Post_author' : []
                  }

    # Loop for saving post details
    for post in posts:
        # print(post)
        # Title of each post
        posts_dict["Title"].append(post.title)

        # Text inside a post
        posts_dict["Post Text"].append(post.selftext)

        # Unique ID of each post
        posts_dict["ID"].append(post.id)

        # The score of a post
        posts_dict["Score"].append(post.score)

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(post.num_comments)

        # Author of the post
        posts_dict['Post_author'].append(post.author)

        # URL of each post
        # print('https://www.reddit.com'+f'{post.permalink}')
        posts_dict["Post URL"].append('https://www.reddit.com'+f'{post.permalink}')
        
    return posts_dict

### Examples of how scraping top month posts works

In [5]:
dict_ = scrape_top_month('politics')

C:\Users\aidan\AppData\Local\Temp\ipykernel_1700\1251007598.py:6: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  posts = subreddit.top("month", limit=ppsr)


In [6]:
# post samples
print(dict_['Title'][0])
print(dict_['Post Text'][0])
print(dict_['ID'][0])
print(dict_['Score'][0])
print(dict_['Total Comments'][0])
print(dict_['Post URL'][0])
print(len(dict_['Title']))
print(dict_['Post_author'][0])

Biden to propose 25% billionaire tax

11mgaw2
98322
6016
https://www.reddit.com/r/politics/comments/11mgaw2/biden_to_propose_25_billionaire_tax/
100
bhodrolok


In [7]:
dict_ = scrape_top_month('politics', ppsr=150)

C:\Users\aidan\AppData\Local\Temp\ipykernel_1700\1251007598.py:6: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  posts = subreddit.top("month", limit=ppsr)


In [9]:
# post samples
print(dict_['Title'][0])
print(dict_['Post Text'][0])
print(dict_['ID'][0])
print(dict_['Score'][0])
print(dict_['Total Comments'][0])
print(dict_['Post URL'][0])
print(len(dict_['Title']))
print(dict_['Post_author'][0])

Biden to propose 25% billionaire tax

11mgaw2
98313
6016
https://www.reddit.com/r/politics/comments/11mgaw2/biden_to_propose_25_billionaire_tax/
150
bhodrolok


## Getting comments on top monthly posts on multiple subreddits

Data we are keeping and why:
 - post_title: to embed and become node values of our users
 - post_url: in case we ever want to visit the post for any inspection reasons
 - comment_author: to keep track of who left the comment
 - comment_text: to analyse the sentiment of the comment towards the post
 - post_author: to track who made the post

In [57]:
def scrape_multiple_save(subreddits, ppsr=100, n=100, save=False, destination=''):
    '''scrapes and saves subreddit comments to csv files
     - subreddits: list of subreddit name strings, fx Politics
     - ppsr: number of posts per subbreddit to scrape
     - n: number of comments per post to scrape'''
    
    if save==False:
        print('WARNING: save=True has not been specified!')

    print(f'Scraping {ppsr} posts per subreddit and {n} comments per post')
    
    # looping through subreddits
    for subreddit in subreddits:
        print(f'Scraping r/{subreddit}...')
        
        # initialize dictionary for saving all comments and post info
        sub_dict = {'post_title': [],
                    # 'post_text': [],
                    # 'post_id': [],
                    # 'post_score': [],
                    # 'post_total_comments': [],
                    'post_url': [],
                    'comment_author': [],
                    'comment_text': [], 
                    'post_author' : []}
        
        posts_dict = scrape_top_month(subreddit, ppsr) # getting top of the month post info
        
        # looping through posts
        for idx, url in tqdm(enumerate(posts_dict['Post URL']),):
            
            # df for comments on the post
            comment_df = scrape_post(url, n=n)
            
            # looping through comments on post and appending all comment info to sub_dict
            for row_idx, row in comment_df.iterrows():
                sub_dict['post_title'].append(posts_dict['Title'][idx])
                # sub_dict['post_text'].append(posts_dict['Post Text'][idx])
                # sub_dict['post_id'].append(posts_dict['ID'][idx])
                # sub_dict['post_score'].append(posts_dict['Score'][idx])
                # sub_dict['post_total_comments'].append(posts_dict['Total Comments'][idx])
                sub_dict['post_url'].append(posts_dict['Post URL'][idx])
                sub_dict['comment_author'].append(row['author'])
                sub_dict['comment_text'].append(row['comment'])
                sub_dict['post_author'].append(posts_dict['Post_author'][idx])
            
        # changing sub_dict to pandas dataframe
        global sub_df
        sub_df = pd.DataFrame.from_dict(sub_dict)

        # saving to csv
        if save==True:
            sub_df.to_csv(f'{destination}{subreddit}.csv', index=False)
        
    print('Done!')

    if save==True:
        return None
    else:
        return sub_df

### How to use function

Below line shows how to scrape r/politics, scraping 200 posts and 100 comments from each post.

It won't save the dataframe unless save=True is specified, so that it can be tested without overwriting anything.

It takes ~ 30-40 minutes to run as it's scraping 200*100 = 20,000 comments. It won't end up actually being 20,000 comments however, probably because 200 posts down there will be less than 100 top-level comments.

In [42]:
# takes ~ 40 minutes to run (it's scraping 200*100 = 20,000 comments - though it wont be that many)
df_test = scrape_multiple_save(['politics'], ppsr=200, n=100)

Scraping 200 posts per subreddit and 100 comments per post
Scraping r/politics...


C:\Users\aidan\AppData\Local\Temp\ipykernel_1700\1251007598.py:6: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  posts = subreddit.top("month", limit=ppsr)
200it [34:22, 10.31s/it]

Done!


In [43]:
print(df_test.shape)
df_test.head()

(19340, 5)


,post_title,post_url,comment_author,comment_text,post_author
0,Biden to propose 25% billionaire tax,https://www.reddit.com/r/politics/comments/11m...,AutoModerator,"\nAs a reminder, this subreddit [is for civil ...",bhodrolok
1,Biden to propose 25% billionaire tax,https://www.reddit.com/r/politics/comments/11m...,rounder55,But that'll impact the 7th generation of my li...,bhodrolok
2,Biden to propose 25% billionaire tax,https://www.reddit.com/r/politics/comments/11m...,LudovicoSpecs,Now we're talking.\n\nStop going after the peo...,bhodrolok
3,Biden to propose 25% billionaire tax,https://www.reddit.com/r/politics/comments/11m...,amorousambrosia,You know who's gonna be the most pissed off ab...,bhodrolok
4,Biden to propose 25% billionaire tax,https://www.reddit.com/r/politics/comments/11m...,atxlrj,"Here in Texas, they have a Vehicle Asset Test ...",bhodrolok


In [44]:
testy = df_test.copy()

In [45]:
testy.columns

Index(['post_title', 'post_url', 'comment_author', 'comment_text',
       'post_author'],
      dtype='object')

In [46]:
print(testy['post_title'][0])

Biden to propose 25% billionaire tax


In [51]:
print(testy['post_url'][0])

https://www.reddit.com/r/politics/comments/11mgaw2/biden_to_propose_25_billionaire_tax/


In [52]:
print(testy['comment_author'][0])

AutoModerator


In [53]:
print(testy['comment_text'][0])


As a reminder, this subreddit [is for civil discussion.](/r/politics/wiki/index#wiki_be_civil)

In general, be courteous to others. Debate/discuss/argue the merits of ideas, don't attack people. Personal insults, shill or troll accusations, hate speech, any suggestion or support of harm, violence, or death, and other rule violations can result in a permanent ban. 

If you see comments in violation of our rules, please report them.

 For those who have questions regarding any media outlets being posted on this subreddit, please click [here](https://www.reddit.com/r/politics/wiki/approveddomainslist) to review our details as to our approved domains list and outlet criteria.
 

***


*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/politics) if you have any questions or concerns.*


In [54]:
print(testy['post_author'][0])

bhodrolok


### Running scrapes - ONLY RUN IF WILLING TO LEAVE RUNNING FOR HOURS

Make sure to specify destination folder before running. Example destination: destination='../data/28feb/scrapes/'

In [58]:
# scrape_multiple_save(['politics'], ppsr=200, n=100, save=True, destination='../data/15march/scrapes/')